# SQL Agent — Natural Language to SQL Query Generator

Welcome to the **SQL Agent** project — an AI-powered assistant that translates natural language questions into executable SQL queries using the schema of your database as context.

This notebook serves both as a **functional prototype** and a **well-structured codebase simulation**, where each cell mirrors a standalone module (as if it were a separate `.py` file in a production system). The goal is to present not only the technical capabilities but also good software engineering practices — even in an environment like Google Colab.

---

## Project Description

**SQL Agent** allows users to:
- Upload their database schema in JSON format,
- Ask questions in natural language,
- Receive accurate SQL queries using an LLM (e.g., Google Gemini),
- Optionally validate the queries syntactically and structurally.

---

## Technologies Used

- **Google Gemini API** (via `google.generativeai`)
- **Python** (Colab environment)
- **Modular code-style cells** (each cell = production-ready `.py` module)
- **[Future-ready]** Vector search (e.g., ChromaDB), LLM tools (Ollama), SQL validators, MLflow, FastAPI, Docker

---

## Module Overview

| Cell (Simulated File)         | Description |
|------------------------------|-------------|
| `01_schema_loader.py`        | Uploads and parses database schema from JSON |
| `02_prompt_builder.py`       | Builds LLM prompt using schema + user query |
| `03_llm_agent.py`            | Sends prompt to Gemini LLM and returns SQL |
| `04_sql_validator.py`        | (Optional) Validates SQL syntax / explain plans |
| `05_interactive_loop.py`     | Interactive CLI-style querying in notebook |
| `99_readme_and_architecture.md` | Documentation of architecture, tools, design |

---

## Evaluation (to be added)

Later we can include:
- SQL correctness tests against mock data
- Query benchmarking via `EXPLAIN`
- Error handling and fallback logic

---

## Deployment Notes

If we had the computational resources (e.g., cloud VM, GPU), this would be deployed as:
- A FastAPI microservice,
- Containerized with Docker,
- Using MLflow to track prompt engineering experiments and LLM outputs,
- Optionally serving open-source LLMs locally with [Ollama](https://ollama.com/).

---

## 📁 Example Repo Structure (Future)

ql-agent/
*  app/
** main.py # FastAPI app
** prompt_builder.py # Prompt creation logic
** schema_loader.py # JSON parsing and formatting
** llm_agent.py # Model abstraction (Gemini, GPT, etc.)
** validator.py # SQL validation logic
* models/ # Precomputed embeddings or cached responses
* tests/ # Unit tests
* Dockerfile
* README.md

Let's get started by uploading your database schema in the next cell.

# 1. schema_loader.py        

 Uploads and parses database schema from JSON

In [ ]:
import json
from google.colab import files # We use it only as a option to make easy the upload of the .json file

def load_schema_from_json():
  """ A function that ask from the client
  the json file and creates the schema
  to parse in into the promt later
  Inputs:
  Outputs:
    schema_info: the schema as the information that will pass to the prompt
    schema_data: the raw file that the user upload """

  # Upload the .json file
  print('Upload the .json file:')
  uploaded = files.upload()

  # Check for errors
  if not uploaded:
    print('No file uploaded from the user.')
    return None, None

  file_name = list(uploaded.keys())[0]
  try:
    with open(file_name, 'r') as f:
      schema_data = json.load(f)
  except json.JSONDecodeError as e:
    print('Invalid JSON format.')
    return None, None
  except Exception as e:
    print(f"Error while reading the file: {e}")
    return None, None

  # We have a dict, that we have to make it into str to parse it into LLM
  schema_info = ""
  if 'tables' in schema_data:
    for table in schema_data['tables']:
      schema_info += f"Table: {table['name']}\n"
      for column in table.get('columns', []):
        schema_info += f" {column['name']} {column['type']}\n"
        schema_info += "\n"
  else:
    print("JSON file does not have the expected 'tables' structure.")
    return None, None

  return schema_data, schema_info

## 2. prompt_builder.py

 Builds LLM prompt using schema + user query

In [ ]:
def prompt_builder(schmema_info: str, user_question: str) -> str:
  """ The prompt that we will pass to the LLM
      Inputs:
        schemma_info: The json schemma into str
        user_question: The physical language question from the client
      Outputs:
        prompt: The prompt that we will pass to the LLM
  """

  prompt = f"""
  You are a SQL expert. Given the following database schema:
  {schema_info}
  answer the following question:
  {user_question}
  by returning a well formed SQL query, as a raw string and no a markdown.
  You will only return the SQL query, nothing else.
  """
  return prompt


In [ ]:
user_question = 'Give me the list of people who order more than 3 items in a single order.'
prompt = prompt_builder(schema_info, user_question)
print(prompt)


  You are a SQL expert. Given the following database schema:
  Table: products
 product_id INTEGER

 product_name TEXT

 price REAL

 stock_quantity INTEGER

Table: customers
 customer_id INTEGER

 customer_name TEXT

 email TEXT

 signup_date TEXT

Table: orders
 order_id INTEGER

 customer_id INTEGER

 order_date TEXT

 total_amount REAL

Table: order_items
 order_item_id INTEGER

 order_id INTEGER

 product_id INTEGER

 quantity INTEGER

 unit_price REAL


  answer the following question:
  Give me the list of people who order more than 3 items in a single order.
  by returning a well formed SQL query, as a raw string and no a markdown.
  You will only return the SQL query, nothing else.
  


## 3. llm_agent.py
  
  Sends prompt to Gemini LLM and returns SQL

In [ ]:
# Create a rull to check the response from the LLM.
# A basic rull is the first word of the query to be on of these words
VALID_SQL_STARTS = ("select", "insert", "update", "delete", "create", "drop", "alter", "with", "explain")

In [ ]:
import re

# Also fix the markdown response of the llm
def clean_sql_output(raw_text):
    # Remove Markdown-style code block markers and language hints
    cleaned = re.sub(r"```(?:sql)?", "", raw_text, flags=re.IGNORECASE)
    cleaned = cleaned.replace("```", "").strip()
    return cleaned

In [ ]:
import google.generativeai as genai
from google.colab import userdata

def init_LLM(api_key: str, model_name: str) -> bool:
  """ Check if the API Key is valid and initiate the model with the model name
    Inputs:
      api_key: Google API Key
      model_name: The name of the model that we want to use
    Outputs:
      True if the authentication is successful, False otherwise
      Model object if the authentication is successful, None otherwise
  """
  try:
    # Load the api key throw secrets
    api_key = userdata.get('GOOGLE_API_KEY')
    if not api_key:
      raise ValueError("API key not found. Please set it in Google Secrets.")

    genai.configure(api_key=api_key)
    model = genai.GenerativeModel(model_name)
    return True, model
  except Exception as e:
    print(f"Authentication failed: {e}")
    return False, None

def generate_sql_query(prompt: str, model_name: str = 'gemini-1.5-flash') -> str:
  """ Generate the SQL query from the prompt
    Inputs:
      prompt: The prompt that we want to pass to the LLM
      model_name: The name of the model that we want to use
    Outputs:
      sql_query: The SQL query generated by the LLM
  """

  # Initiate the LLM model
  try:
    auth, model = init_LLM(api_key, model_name)

  except Exception as e:
    print(f"Error while initializing the LLM: {e}")
    return None

  # Create the response
  try:
    response = model.generate_content(prompt)
    raw_output = response.text
    sql_query = clean_sql_output(raw_output)

    # Check the formality of the response as a safety key
    #if sql_query.lower().startswith(VALID_SQL_STARTS):
    #  print('Warning: The response is not a valid SQL statement.')

    return sql_query
  except Exception as e:
    print(f"Error while generating the SQL query: {e}")
    return ""

## 4. sql_validator.py (On process)
  Validates SQL syntax / explain plans with the help of the `sqlite3` library

In [ ]:
import sqlite3

def _generate_create_statement(schema_info: str) -> list:
  """ Generates from the schema_info a list of create statements of SQL Table
    Inputs:
      schema_info: The json schemma into str
    Outputs:
      create_statements: The list of create statements
  """

  statements = []
  tables = schema_info.strip().split('Table: ')[1:]

  for table in tables:
    lines = table.strip().split('\n')
    table_name = lines[0].strip()
    columns = lines[1:]

    col_defs = []
    for col in columns:
      parts = col.ctrip().split()
      col_name = parts[0]
      col_type = parts[1] if len(parts) > 1 else 'TEXT'
      col_defs.append(f'{col_name} {col_type}')

    create_statement = f'CREATE TABLE {table_name} ({", ".join(col_defs)})'
    statements.append(create_statement)

  return statements


def validate_sql_syntax(sql_query: str, schema_info: str) -> bool:
  """ A function that checks if the query is valid.
    Inputs:
      sql_query: The SQL query that we want to validate
      schema_info: The json schemma into str
    Outputs:
      True if the query is valid, False otherwise
  """

  try:
    # Create a memory in the base
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    # Create a schema based on the schema_info
    create_statements = _generate_create_statement(schema_info)
    for st in create_statements:
      cursor.execute(st)

    # Try to run the query
    cursor.execute(f"EXPLAIN {sql_query}")
    conn.close()
    print('SQL syntax is valid')
    return True

  except Exception as e:
    print(f'SQL syntax error: {e}')
    return False


## 5. interactive_loop.py`

Interactive CLI-style querying in notebook  

In [ ]:
# Load the schema (simulates schema_loader.py)
schema_data, schema_info = load_schema_from_json()

Upload the .json file:


Saving toy.json to toy (1).json


In [ ]:
# Exit if schema not loaded
if not schema_info:
    print("Schema loading failed. Exiting.")
else:
    print("Schema loaded successfully!\n")
    print(schema_info)

    # Start interactive loop
    print("\n Ask your question in natural language (type 'quit' to exit):")
    while True:
        user_question = input()
        if user_question.lower() in ['quit', 'exit']:
            print("Exiting the SQL agent. Goodbye!")
            break

        # Build prompt
        prompt = prompt_builder(schema_info, user_question)

        # Generate SQL query
        sql_query = generate_sql_query(prompt)

        # Output result
        if sql_query:
            print("\n Generated SQL Query:")
            print(sql_query)
            print('\n Ready for the next question or type "quit" to exit')
        else:
            print("Could not generate a valid SQL query.")
            print('\n Ready for the next question or type "quit" to exit')

Schema loaded successfully!

Table: products
 product_id INTEGER

 product_name TEXT

 price REAL

 stock_quantity INTEGER

Table: customers
 customer_id INTEGER

 customer_name TEXT

 email TEXT

 signup_date TEXT

Table: orders
 order_id INTEGER

 customer_id INTEGER

 order_date TEXT

 total_amount REAL

Table: order_items
 order_item_id INTEGER

 order_id INTEGER

 product_id INTEGER

 quantity INTEGER

 unit_price REAL



 Ask your question in natural language (type 'quit' to exit):
tell me the names of the best clients

 Generated SQL Query:
SELECT c.customer_name
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_name
ORDER BY SUM(o.total_amount) DESC
LIMIT 5;

 Ready for the next question or type "quit" to exit
thnaks, quit

 Generated SQL Query:
-- No query needed as the request is "thanks, quit"

 Ready for the next question or type "quit" to exit
quit
Exiting the SQL agent. Goodbye!


# Conclusion & Next Steps

This SQL Agent prototype successfully demonstrates how LLMs can convert natural language into SQL. Next steps could include:

- Connecting to more complex databases.
- Adding validation or feedback loops.
- Integrating vector-based retrieval (see next notebook).
